In [5]:
# copy from main_draft-1sted | intended to copy all maniplution of dataset and 
# relevant vairables in order to reduse clus when implimented in finale draft

import warnings # Got an irritating warning
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows", None)
pd.set_option("float_format", "{:f}".format)

aron_path = "H:\Informasjonsvitenskap\Programming\Python\Info-284\Info284_Project\Exam Task\Dataset\elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv"
sverre_path = r"C:\Users\Sverre\Documents\VS Studio. filer\INFO284\Obli\Info284_Project\Exam Task\Dataset\elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv"

dataset = pd.read_csv(sverre_path, sep = ";")

# Dataset where the species isn"t the same as the main-species
bycatch = dataset[dataset["Art FAO"] != dataset["Hovedart FAO"]]

# Dataset where the species is the same as the main-species
main_species = dataset[dataset["Art FAO"] == dataset["Hovedart FAO"]]

# Excluding all features that aren't relevant
dataset = dataset[["Hovedart FAO", "Art FAO", "Lengdegruppe", "Redskap FAO", "Rundvekt", "Hovedområde start"]]

# Drop all rows where Hovedart FAO and Art FAO is NaN
dataset = dataset.dropna(subset=["Hovedart FAO", "Art FAO"])

# Make new category in lengdegruppe
dataset["Lengdegruppe"] = dataset["Lengdegruppe"].fillna("Stortare båter")

# Drop Na values
dataset = dataset.dropna(subset=["Redskap FAO"])

# Dropping Na values
dataset = dataset.dropna(subset=["Hovedområde start"])

# Create a new binary feature to use as target variable. 
dataset["Is_Bycatch"] = (dataset["Hovedart FAO"] != dataset["Art FAO"])

# ==========================================================================|

# Changing categorical data into numeric data using "cat.codes"
dataset["Art FAO"] = dataset["Art FAO"].astype("category")
dataset["Art FAO Codes"] = dataset["Art FAO"].cat.codes

dataset["Lengdegruppe"] = dataset["Lengdegruppe"].astype("category")
dataset["Lengdegruppe Codes"] = dataset["Lengdegruppe"].cat.codes

dataset["Hovedområde start"] = dataset["Hovedområde start"].astype("category")
dataset["Hovedområde start Codes"] = dataset["Hovedområde start"].cat.codes 

dataset["Redskap FAO"] = dataset["Redskap FAO"].astype("category")
dataset["Redskap FAO Codes"] = dataset["Redskap FAO"].cat.codes 

# Excluding the old features
dataset = dataset[["Art FAO Codes", "Lengdegruppe Codes", "Redskap FAO Codes", 
                   "Rundvekt", "Hovedområde start Codes", "Is_Bycatch"]]


from sklearn.model_selection import train_test_split

# Splitting dataset 
X = dataset[["Art FAO Codes", "Lengdegruppe Codes", "Redskap FAO Codes", "Rundvekt", "Hovedområde start Codes"]]
y = dataset["Is_Bycatch"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer

# Scaling only "Rundvekt" and not the others
ct = ColumnTransformer(
    [("scale", RobustScaler(), ["Rundvekt"])],
    remainder="passthrough"
)

# Fitting the scaler to the training set and not the test set to prevent data leakage
X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.transform(X_test)


from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import precision_recall_curve
from sklearn.datasets import make_blobs
from sklearn.pipeline import Pipeline

## Our choices
- Neural network
"Neural networks have a considerable succsess in low-level reasoning for with there is aboundant training data ... one reason is that they are very flexible and can invent features ... As far as learning is conserned, neural network provide a different measure of simplicity as a learning bias then, for eksample, decision trees. Multilayer neural networks, like decision tree, can represent any function of a set of discreat features." [Alan K.Mackworth, David L. Poole (2017). p.309.  "Artificial intelligence, Foundations of computational agents". Cambridge, United Kingdom: Cambridge University Press]

### Neural Network

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV


A basic implementation of a neural network and something to compare the testing and fine tuning to come. 

In [3]:

mlp = MLPClassifier(verbose=False, hidden_layer_sizes=(10, 5), learning_rate_init=0.001, random_state=50)

mlp.fit(X_train_scaled, y_train)

y_test_pred = mlp.predict(X_train_scaled)
train_accuracy = accuracy_score(y_test_pred, y_train)
print("Training accuracy: ", train_accuracy)


y_pred = mlp.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print(f"Confusion Matrix: {confusion_matrix(y_test, y_pred)}")


Training accuracy:  0.8746430290029097
Accuracy: 0.8734754452239611
Confusion Matrix: [[17278  6009]
 [ 3369 47464]]


In [8]:

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import precision_recall_curve
from sklearn.datasets import make_blobs
from sklearn.pipeline import Pipeline

We are to check if the neural network model is vulnerable to a potential fluctuation in where the train test split is happening. 

In [ ]:
skf = StratifiedKFold(n_splits=10)

training_acc_list = []
testing_acc_list =[]
con_x_list = []
o_u_fitting_list= []


for train_index, test_index in skf.split(X, y):

    X_train_fold = X.iloc[train_index]
    X_test_fold = X.iloc[test_index]
    y_train_fold = y.iloc[train_index]
    y_test_fold = y.iloc[test_index]

    X_train_fold_scaled = ct.fit_transform(X_train_fold)
    X_test_fold_scaled = ct.transform(X_test_fold)

    mlp.fit(X_train_fold_scaled, y_train_fold)

    training_acc_list.append(mlp.score(X_train_fold_scaled, y_train_fold))
    testing_acc_list.append(mlp.score(X_test_fold_scaled, y_test_fold))
    con_x_list.append(confusion_matrix(y_test_fold, mlp.predict(X_test_fold_scaled)))
    train_test_diff = (mlp.score(X_train_fold_scaled, y_train_fold)) - (mlp.score(X_test_fold_scaled, y_test_fold))
    o_u_fitting_list.append(train_test_diff)


print(training_acc_list, "\n", 
      testing_acc_list, "\n", 
      con_x_list, "\n", 
      o_u_fitting_list)



[0.8968552893426127, 0.8973499881946865, 0.8959221074171099, 0.8935722878697593, 0.9008503573449662, 0.8846789516881598, 0.8973125110089233, 0.8975564966458044, 0.9021361915826557, 0.8933927969118914] 
 [0.8928764166216946, 0.9208378305450621, 0.9098421478683216, 0.8818470048569886, 0.9028939557474366, 0.889840798704803, 0.8842754991905019, 0.8568489223192903, 0.8745572907882754, 0.8949640773096772] 
 [array([[ 7595,  1658],
       [ 1518, 18877]], dtype=int64), array([[ 8366,   887],
       [ 1460, 18935]], dtype=int64), array([[ 8107,  1146],
       [ 1527, 18868]], dtype=int64), array([[ 7304,  1949],
       [ 1554, 18841]], dtype=int64), array([[ 7399,  1854],
       [ 1025, 19370]], dtype=int64), array([[ 7240,  2013],
       [ 1253, 19142]], dtype=int64), array([[ 7223,  2030],
       [ 1401, 18994]], dtype=int64), array([[ 6201,  3052],
       [ 1192, 19202]], dtype=int64), array([[ 6438,  2814],
       [  905, 19490]], dtype=int64), array([[ 7392,  1860],
       [ 1254, 19141]]

'\nfor i, (train_index, test_index) in enumerate(skf.split(X, y)):\n    ct = ColumnTransformer(\n        [("scale", RobustScaler(), ["Rundvekt"])],\n        remainder="passthrough")\n    X_train_scaled = ct.fit_transform(X_train)\n    X_test_scaled = ct.transform(X_test)\n\n    # aplying split to algo\n    mlp = MLPClassifier(verbose=False, hidden_layer_sizes=(10, 5), learning_rate_init=0.001, random_state=50)\n\n    mlp.fit(X_train_scaled, y_train)\n\n    y_test_pred = mlp.predict(X_train_scaled)\n    train_accuracy = accuracy_score(y_test_pred, y_train)\n    training_acc_list.append(train_accuracy)\n\n    y_pred = mlp.predict(X_test_scaled)\n    accuracy = accuracy_score(y_test, y_pred)\n    testing_acc_list.append(accuracy)\n\n    o_u_fitting_list.append(accuracy - train_accuracy)\n    con_x_list.append(confusion_matrix(y_test, y_pred))\n'

The output's second line highlights a significant gap in prediction performance, with nearly a 7% point difference between the lowest (~85.6) and highest (~92.0) correct prediction rates. This suggests that the predictor is sensitivity to how the data splitting occurs. Without shuffling, it may indicate a concentration of certain data aspects, while with sufficient shuffling, it may reveal more entries with a specific label.

In 8/10 interactions, the confusion matrix shows a higher number of false positives than false negatives. But the values of the confusion matrix appear to be somewhat volatile and fluctuate within a range of a few hundred, so this is something to keep in mind for the future. 


In the last line of the output, you can see the difference between prediction accuracy on the training and test data. Half of the numbers are negative, indicating that in this cross-validation iteration, the model performs worse on the training data than on the test data. Possible reasons for this could include data leakage, significant differences in the data splitting, or a model structure that isn't complex enough to fit the training data effectively.


To find the best parameters for the neural network, we conducted a grid search. Early stopping was enabled to use a portion of the training dataset as a validation set, stopping training if no improvement is observed. Cross-validation with F1 scoring and pipeline were employed to address potential issues mentioned earlier.

In [ ]:

parameters = {"neural_net__activation": ["identity", "logistic", "tanh", "relu"],
              "neural_net__solver": ["lbfgs", "sgd", "adam"],
              "neural_net__learning_rate": ["constant", "invscaling", "adaptive"],
              "neural_net__learning_rate_init":[0.01, 0.001, 0.0001],
              "neural_net__hidden_layer_sizes": [(100,), (50,), (10,), (5,), (1,),  
                                                 (100, 50), (50, 50), (10, 5), (4, 2), (2, 2), ]      
} 

mlp = MLPClassifier(early_stopping=True, random_state=50)

pipe = Pipeline([("scaler", ct), ("neural_net", mlp)])

grid = GridSearchCV(pipe, 
                    param_grid=parameters, 
                    cv=10, 
                    verbose=1, 
                    n_jobs=-1,
                    scoring=make_scorer(f1_score))

grid.fit(X_train, y_train)

print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))
best_model = grid.best_estimator_

#[output] >>>

"""
Fitting 10 folds for each of 1512 candidates, totalling 15120 fits

Best cross-validation accuracy: 0.94
Test set score: 0.94
Best parameters: {'neural_net__activation': 'tanh', 
'neural_net__hidden_layer_sizes': (100, 50), 
'neural_net__learning_rate': 'constant', 
'neural_net__learning_rate_init': 0.001, 
'neural_net__solver': 'adam'}

"""

After runing the python cell above we got the following output:

[output] >>>

Fitting 10 folds for each of 1512 candidates, totalling 15120 fits

Best cross-validation accuracy: 0.94
Test set score: 0.94
Best parameters: {'neural_net__activation': 'tanh', 
'neural_net__hidden_layer_sizes': (100, 50), 
'neural_net__learning_rate': 'constant', 
'neural_net__learning_rate_init': 0.001, 
'neural_net__solver': 'adam'}
___________________________________________________

After grid searching some parameters, we found that the above parameters yielded the best model performance. However, fine-tuning only resulted in a 2% point improvement compared to the best cross-validation integration for the basic implementation. This change is relatively small, especially considering the larger impact the data splitting had on performance. It's possible that this minor improvement is due to the high accuracy of the model, and the remaining mispredictions are deviating slightly from the overall pattern.

To gain the final percentage points, further fine-tuning of the hidden layer is likely necessary. The output indicates that the most complex model performed the best, suggesting that the current model might be too simplistic to capture the underlying pattern effectively. Further testing is needed to confirm this hypothesis.


Using the same parameters as with the best performing model from the gridsearch, we made some changes to how the hidden layers were defined. 

In [13]:
arameters = {"neural_net__activation": ["tanh"],
              "neural_net__solver": ["adam"],
              "neural_net__learning_rate": ["constant"],
              "neural_net__learning_rate_init":[0.001],
              "neural_net__hidden_layer_sizes": [(100, 50), (100, 100),
                                                 (10, 10, 10), (50, 50, 50), (100, 100, 100),
                                                 (50, 50, 50, 50)]      
} 

mlp = MLPClassifier(early_stopping=True, random_state=50)

pipe = Pipeline([("scaler", ct), ("neural_net", mlp)])

grid = GridSearchCV(pipe, 
                    param_grid=arameters, 
                    cv=10, 
                    verbose=1, 
                    n_jobs=-1,
                    scoring=make_scorer(f1_score))

grid.fit(X_train, y_train)

print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))
best_model = grid.best_estimator_

Fitting 10 folds for each of 6 candidates, totalling 60 fits
Best cross-validation accuracy: 0.95
Test set score: 0.95
Best parameters: {'neural_net__activation': 'tanh', 'neural_net__hidden_layer_sizes': (100, 100, 100), 'neural_net__learning_rate': 'constant', 'neural_net__learning_rate_init': 0.001, 'neural_net__solver': 'adam'}


[output] >>> 

Fitting 10 folds for each of 6 candidates, totalling 60 fits

Best cross-validation accuracy: 0.95

Test set score: 0.95

Best parameters: {'neural_net__activation': 'tanh', 'neural_net__hidden_layer_sizes': (100, 100, 100), 'neural_net__learning_rate': 'constant', 'neural_net__learning_rate_init': 0.001, 'neural_net__solver': 'adam'}

_______________________________________________________________

The model's 1% point improvement confirms our theory. Despite an alternative parameter with more hidden layers but fewer neurons, it may not have been a more complex or suitable model. If we were to wish to improve the model any more in the future, a lot more exploring and fine tuning on this aspect of the model would be needed. 